In [34]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score

In [11]:
train1=pd.read_csv('lightGBM_out_of_fold_train_predictions.csv')

In [12]:
train1.shape

(307511, 3)

In [13]:
train1.head()

,SK_ID_CURR,fold_id,lightGBM_prediction
0,100002,0,0.304156
1,100009,0,0.005594
2,100011,0,0.049608
3,100014,0,0.045775
4,100017,0,0.018538


In [14]:
test1=pd.read_csv('lightGBM_out_of_fold_test_predictions.csv')

In [15]:
test1.shape

(243720, 3)

In [16]:
test1.head()

,SK_ID_CURR,fold_id,lightGBM_prediction
0,100001,0,0.046421
1,100005,0,0.113959
2,100013,0,0.036334
3,100028,0,0.046933
4,100038,0,0.159083


In [17]:
set(train1['fold_id'])

{0, 1, 2, 3, 4}

In [18]:
train2=pd.read_csv('out_folder_train_prediction_lgb.csv')

In [19]:
train2.shape

(307511, 4)

In [20]:
train2.head()

,train_pred,SK_ID_CURR,TARGET,folder
0,0.039001,100004,0.0,1
1,0.044883,100006,0.0,1
2,0.050955,100008,0.0,1
3,0.057840,100011,0.0,1
4,0.009610,100018,0.0,1


In [21]:
test2=pd.read_csv('out_folder_test_prediction_lgb.csv')

In [22]:
test2.shape

(243720, 3)

In [23]:
test2.head()

,SK_ID_CURR,test_pred,folder
0,100001,0.035022,1
1,100005,0.149451,1
2,100013,0.046266,1
3,100028,0.035332,1
4,100038,0.169767,1


In [27]:
train3=pd.read_csv('out_folder_train_prediction_xgb.csv')

In [28]:
train3.shape

(307511, 4)

In [29]:
train3.head()

,train_pred,SK_ID_CURR,TARGET,folder
0,0.033511,100004,0.0,1
1,0.042953,100006,0.0,1
2,0.056619,100008,0.0,1
3,0.052434,100011,0.0,1
4,0.007602,100018,0.0,1


In [30]:
test3=pd.read_csv('out_folder_test_prediction_xgb.csv')

In [31]:
test3.shape

(243720, 3)

In [32]:
test3.head()

,SK_ID_CURR,test_pred,folder
0,100001,0.029586,1
1,100005,0.209405,1
2,100013,0.043811,1
3,100028,0.034501,1
4,100038,0.188071,1


In [33]:
dfs = [train1,train2,train3]

In [35]:
train_final = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)

In [40]:
train_final.shape

(307511, 9)

In [36]:
train_final.head()

,SK_ID_CURR,fold_id,lightGBM_prediction,train_pred_x,TARGET_x,folder_x,train_pred_y,TARGET_y,folder_y
0,100002,0,0.304156,0.292024,1.0,2,0.272052,1.0,2
1,100009,0,0.005594,0.004558,0.0,3,0.005636,0.0,3
2,100011,0,0.049608,0.057840,0.0,1,0.052434,0.0,1
3,100014,0,0.045775,0.038774,0.0,2,0.046881,0.0,2
4,100017,0,0.018538,0.023431,0.0,3,0.021113,0.0,3


In [39]:
X_train=train_final[['lightGBM_prediction','train_pred_x','train_pred_y' ]]
y_train=train_final['TARGET_x']

In [41]:
stack_model = LogisticRegression()
stack_model.fit(X_train, y_train)
stack_model.coef_

array([[ 3.72164124,  3.26593421,  1.2288805 ]])

In [42]:
stack_model.score(X_train, y_train)

0.91964840282136251

In [43]:
dfs = [test1[test1['fold_id']==0],test2[test2['folder']==1],test3[test3['folder']==1]]

In [44]:
test_final1 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)

In [46]:
test_final1.shape

(48744, 7)

In [47]:
test_final1.head()

,SK_ID_CURR,fold_id,lightGBM_prediction,test_pred_x,folder_x,test_pred_y,folder_y
0,100001,0,0.046421,0.035022,1,0.029586,1
1,100005,0,0.113959,0.149451,1,0.209405,1
2,100013,0,0.036334,0.046266,1,0.043811,1
3,100028,0,0.046933,0.035332,1,0.034501,1
4,100038,0,0.159083,0.169767,1,0.188071,1


In [48]:
pred0=stack_model.predict_proba(test_final1[['lightGBM_prediction','test_pred_x','test_pred_y' ]])[:,1]

In [49]:
dfs = [test1[test1['fold_id']==1],test2[test2['folder']==2],test3[test3['folder']==2]]
test_final2 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred1=stack_model.predict_proba(test_final2[['lightGBM_prediction','test_pred_x','test_pred_y' ]])[:,1]

In [50]:
dfs = [test1[test1['fold_id']==2],test2[test2['folder']==3],test3[test3['folder']==3]]
test_final3 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred2=stack_model.predict_proba(test_final3[['lightGBM_prediction','test_pred_x','test_pred_y' ]])[:,1]

In [51]:
dfs = [test1[test1['fold_id']==3],test2[test2['folder']==4],test3[test3['folder']==4]]
test_final4 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred3=stack_model.predict_proba(test_final4[['lightGBM_prediction','test_pred_x','test_pred_y' ]])[:,1]

In [54]:
dfs = [test1[test1['fold_id']==4],test2[test2['folder']==5],test3[test3['folder']==5]]
test_final5 = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)
pred4=stack_model.predict_proba(test_final5[['lightGBM_prediction','test_pred_x','test_pred_y' ]])[:,1]

In [52]:
sub = pd.DataFrame()
sub['SK_ID_CURR'] = test1[test1['fold_id']==0]['SK_ID_CURR']

In [55]:
sub['pred0']=pred0
sub['pred1']=pred1
sub['pred2']=pred2
sub['pred3']=pred3
sub['pred4']=pred4

In [56]:
sub['TARGET']=(sub['pred0']+sub['pred1']+sub['pred2']+sub['pred3']+sub['pred4'])/5

In [57]:
np.min(sub['TARGET'])

0.033977392798511592

In [58]:
np.max(sub['TARGET'])

0.93411145566468579